In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the dataset
file_path = 'combined_data3.csv'  # Update this path as needed
data = pd.read_csv(file_path)

# Fill NaN values with 0.0
data.fillna(0.0, inplace=True)

# Define the target variable and features
X = data.drop('Uplink_thrpt', axis=1)
y = data['Uplink_thrpt']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Define the model pipeline
pipeline = make_pipeline(
    preprocessor,
    DecisionTreeRegressor(random_state=42)
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the entire dataset
data['Predicted_Uplink_thrpt'] = pipeline.predict(X)

# Function to apply conditions with respect to priority
def filter_with_conditions(data, conditions):
    filtered_data = data.copy()
    
    for condition in conditions:
        if 'Predicted' in condition['column']:
                for condition in conditions:
                    if condition['operation'] == '=':
                        # Handle equality condition for categorical/string columns
                        query_string = f"{condition['column']} == '{condition['value']}'"
                    else:
                        # Handle other conditions (e.g., '<', '>', etc.)
                        query_string = f"{condition['column']} {condition['operation']} {condition['value']}"
        
                    filtered_data = filtered_data.query(query_string)
        else:
            for condition in conditions:
                if condition['operation'] == '=':
                    # Handle equality condition for categorical/string columns
                    query_string = f"{condition['column']} == '{condition['value']}'"
                else:
                    # Handle other conditions (e.g., '<', '>', etc.)
                    query_string = f"{condition['column']} {condition['operation']} {condition['value']}"
        
                filtered_data = filtered_data.query(query_string)
                       
    return filtered_data

# Define the conditions in order of priority
conditions = [
    {'column': 'Uplink_delay', 'operation': '<', 'value': 50},
    {'column': 'Communication_link', 'operation': '=', 'value': 'Duplex'},
    {'column': 'Downlink_delay', 'operation': '<', 'value': 50},
    {'column': 'Uplink_pcktloss', 'operation': '<', 'value': 0.1},
    {'column': 'Downlink_pcktloss', 'operation': '<', 'value': 0.1},
    {'column': 'sta_nrg', 'operation': '<', 'value': 110},
#    {'column': 'Uplink_thrpt', 'operation': '<', 'value': 5},
    {'column': 'Downlink_thrpt', 'operation': '>', 'value': 5}
]

# Apply the conditions
optimal_configurations = filter_with_conditions(data, conditions)

# Drop the Predicted_Uplink_thrpt column from the display
optimal_configurations = optimal_configurations.drop(columns=['Predicted_Uplink_thrpt'])
# Output the results DataFrame to CSV file
optimal_configurations.to_csv('Online_gaming.csv', index=False)

# Print the optimal configurations
print("Optimal Configurations to achieve the required conditions:")
print(optimal_configurations)

Optimal Configurations to achieve the required conditions:
      Communication_link Power_save_mechanism Generated_traffic  \
4027              Duplex                  Mix       Full Buffer   
4878              Duplex                  PSM       Full Buffer   
4883              Duplex                  PSM       Full Buffer   
4907              Duplex               Active           Poisson   
5987              Duplex                  TWT          Periodic   
6352              Duplex                  PSM       Full Buffer   
6500              Duplex               Active          Periodic   
6977              Duplex               Active           Poisson   
8045              Duplex               Active          Periodic   
8454              Duplex               Active           Poisson   
8498              Duplex               Active          Periodic   
9566              Duplex               Active          Periodic   
9919              Duplex               Active          Periodic   
110